<a href="https://colab.research.google.com/github/fromakim/2021Election_Analysis/blob/main/data_prepreocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Dev Environment

In [2]:
!pip install flatten_json

In [23]:
import itertools
import json
import requests
from urllib import parse
from datetime import datetime, timedelta

In [4]:
import numpy as np
import pandas as pd

In [5]:
from flatten_json import flatten

# 2. Import Data

In [27]:
# Import Data
data = []

In [31]:
for i in range(100):
    try:
        f = open(f'./data{i:02}.json', encoding = 'utf-8')
        data.append(json.load(f))
    except Exception as e:
        pass

# 3. Organize User, Media, Tweet

In [32]:
users = pd.concat([pd.DataFrame([flatten(i, root_keys_to_ignore = {'public_metrics', 'entities'}) for i in item['includes']['users']]) for item in data]).reset_index(drop = True)

In [33]:
medias = pd.concat([pd.DataFrame([flatten(i) for i in item['includes']['media']]) if 'media' in item['includes'] else pd.DataFrame() for item in data]).reset_index(drop = True)

In [34]:
tweets = pd.concat([pd.DataFrame([flatten(i, root_keys_to_ignore = {'referenced_tweets', 'public_metrics', 'in_reply_to_user_id', 'attachments', 'entities', 'context_annotations', 'attachments'}) for i in item['includes']['tweets']]) for item in data]).reset_index(drop = True)

# 4. Organize overall Relation

In [35]:
user_relations = pd.concat([pd.json_normalize(d['includes']['users']) for d in data]).reset_index(drop = True)[['id', 'entities.url.urls', 'entities.description.urls', 'entities.description.mentions', 'entities.description.hashtags']]

In [36]:
tweet_relations = pd.concat([pd.json_normalize(d['includes']['tweets']) for d in data])[['referenced_tweets', 'entities.urls', 'in_reply_to_user_id']].reset_index(drop = True)

In [37]:
# https://developer.twitter.com/en/docs/twitter-api/users/lookup/api-reference/get-users
user_relations.columns

Index(['id', 'entities.url.urls', 'entities.description.urls',
       'entities.description.mentions', 'entities.description.hashtags'],
      dtype='object')

In [38]:
# https://developer.twitter.com/en/docs/twitter-api/tweets/lookup/api-reference/get-tweets
tweet_relations.columns

Index(['referenced_tweets', 'entities.urls', 'in_reply_to_user_id'], dtype='object')

# 5. Organize User Relations

In [39]:
relation_url_url = user_relations[['id', 'entities.url.urls']].dropna().explode('entities.url.urls')
relation_url_url = pd.concat([relation_url_url, relation_url_url['entities.url.urls'].apply(pd.Series)], axis = 1)
relation_url_url = relation_url_url.drop(columns = 'entities.url.urls').reset_index(drop = True)

In [40]:
relation_url_description = user_relations[['id', 'entities.description.urls']].dropna().explode('entities.description.urls')
relation_url_description = pd.concat([relation_url_description, relation_url_description['entities.description.urls'].apply(pd.Series)], axis = 1)
relation_url_description = relation_url_description.drop(columns = 'entities.description.urls').reset_index(drop = True)

In [41]:
relation_url = pd.concat([relation_url_url, relation_url_description]).reset_index(drop = True)

In [42]:
relation_mention = user_relations[['id', 'entities.description.mentions']].dropna().explode('entities.description.mentions')
relation_mention = pd.concat([relation_mention, relation_mention['entities.description.mentions'].apply(pd.Series)], axis = 1)
relation_mention = relation_mention.drop(columns = 'entities.description.mentions').reset_index(drop = True)

In [43]:
relation_hashtag = user_relations[['id', 'entities.description.hashtags']].dropna().explode('entities.description.hashtags')
relation_hashtag = pd.concat([relation_hashtag, relation_hashtag['entities.description.hashtags'].apply(pd.Series)], axis = 1)
relation_hashtag = relation_hashtag.drop(columns = 'entities.description.hashtags').reset_index(drop = True)

In [ ]:
# TODO: link username to target user id in relation_mention

,id,start,end,username
0,790232729770086400,84,95,hereshippo
1,2684046276,6,22,yunsuljj_notice
2,218527625,122,130,iianfit
3,880759635816333313,10,20,Chou_crem
4,970942487396102144,4,20,dailyshirley120
...,...,...,...,...
83,292395944,22,32,barry_lee
84,1240916626020552705,7,17,oat_duck3
85,1600295274,118,134,artholic_sketch
86,970942487396102144,4,20,dailyshirley120


12533